# README

Parameters may be defined either by range or by tolerance (absolute or relative). Units from [Pint](https://pint.readthedocs.io/en/stable/) are supported. A default `Pint.UnitRegistry()` is provided to the user as `worstcase.unit`.
* `worstcase.param.byrange(nom, lb, ub, tag="")`
* `worstcase.param.bytol(nom, tol, rel, tag="")`

Both Extreme Value and Monte Carlo analyses are supported, even in the same calculation. The underlying computation is performed on the directed graph of `Parameters`, `Derivatives`, and the single-valued functions between them. This directed graph must be an acyclic computation. Extreme Value Analysis is performed by brute force, so Monte Carlo Analysis will be faster if the directed graph couples many parameters together. When `Derivatives` are coupled together, the analysis method of one `Derivative` may be overruled by one of its descendants.
* `derive.byev(*args, tag="", **kwargs)(func)`
* `derive.bymc(*args, tag="", **kwargs)(func)`

In [1]:
%reload_ext lab_black
from pprint import pprint
from worstcase import config, param, derive, unit

## Example 1: Voltage Dividers and an Amplifier

As a contrived example, we will calculate the worst-case voltage variations by extreme value analysis at the amplifier input terminals, the amplifier output terminal, and at the final output.

<img src="voltage_dividers_and_amplifier.png">

In [2]:
config.n = 2000  # Number of Monte Carlo runs. (default: 5000)
config.sigfig = 5  # Number of significant figures to print. (default: 4)

In [3]:
# Define the parameters of interest. See the diagram above.
VCC = param.byrange(nom=5 * unit.V, lb=4.8 * unit.V, ub=5.1 * unit.V, tag="vcc")
R1 = param.bytol(nom=10 * unit.kohm, tol=0.10, rel=True, tag="r1")
R2 = param.bytol(nom=10 * unit.kohm, tol=0.10, rel=True, tag="r2")

VIN = param.byrange(nom=12 * unit.V, lb=11.9 * unit.V, ub=12.05 * unit.V, tag="vin")
R3 = param.bytol(nom=50 * unit.kohm, tol=0.05, rel=True, tag="r3")
R4 = param.bytol(nom=20 * unit.kohm, tol=0.05, rel=True, tag="r4")

GAIN = param.bytol(nom=1.1, tol=0.1, rel=False, tag="gain")
R5 = param.bytol(nom=10 * unit.kohm, tol=0.01, rel=True, tag="r5")
R6 = param.bytol(nom=10 * unit.kohm, tol=0.01, rel=True, tag="r6")

pprint([VCC, VIN, GAIN, R1, R2, R3, R4, R5, R6])

[vcc: 5 V (nom), 4.8 V (lb), 5.1 V (ub),
 vin: 12 V (nom), 11.9 V (lb), 12.05 V (ub),
 gain: 1.1 (nom), 1 (lb), 1.2 (ub),
 r1: 10 kΩ (nom), 9 kΩ (lb), 11 kΩ (ub),
 r2: 10 kΩ (nom), 9 kΩ (lb), 11 kΩ (ub),
 r3: 50 kΩ (nom), 47.5 kΩ (lb), 52.5 kΩ (ub),
 r4: 20 kΩ (nom), 19 kΩ (lb), 21 kΩ (ub),
 r5: 10 kΩ (nom), 9.9 kΩ (lb), 10.1 kΩ (ub),
 r6: 10 kΩ (nom), 9.9 kΩ (lb), 10.1 kΩ (ub)]


In [4]:
# Define a generic voltage divider function.
def voltage_divider(vtop, vbot, rtop, rbot):
    return vbot + rbot * (vtop - vbot) / (rtop + rbot)

In [5]:
# Define input voltage divider derived parameters.
VNEG = derive.byev(vtop=VCC, vbot=0, rtop=R1, rbot=R2, tag="vneg")(voltage_divider)
VPOS = derive.byev(vtop=VIN, vbot=0, rtop=R3, rbot=R4, tag="vpos")(voltage_divider)

pprint([VNEG, VPOS])

[vneg: 2.5 V (nom), 2.16 V (lb), 2.805 V (ub),
 vpos: 3.4286 V (nom), 3.1622 V (lb), 3.6942 V (ub)]


In [6]:
# Define the amplifier output derived parameter.
@derive.byev(vp=VPOS, vm=VNEG, gain=GAIN, tag="amp")
def AMP(vp, vm, gain):
    return gain * (vp - vm)


print(AMP)

amp: 1.0214 V (nom), 357.24 mV (lb), 1.841 V (ub)


In [7]:
# Define the output voltage divider derived parameter.
VOUT = derive.byev(vtop=VCC, vbot=AMP, rtop=R5, rbot=R6, tag="vout")(voltage_divider)

print(VOUT)

vout: 3.0107 V (nom), 2.6397 V (lb), 3.4066 V (ub)


In [8]:
# Confirm with manual calculation.

vneg_min = VCC.lb * R2.lb / (R1.ub + R2.lb)
assert vneg_min == VNEG.lb

vpos_max = VIN.ub * R4.ub / (R3.lb + R4.ub)
assert vpos_max == VPOS.ub

amp_max = GAIN.ub * (vpos_max - vneg_min)
assert amp_max == AMP.ub

vneg = VCC.ub * R2.lb / (R1.ub + R2.lb)
amp = GAIN.ub * (vpos_max - vneg)
vout_max = amp + (VCC.ub - amp) * R6.ub / (R5.lb + R6.ub)
assert vout_max == VOUT.ub

## Example 2: Unit Conversion Support

In [9]:
speed = param.bytol(10 * unit("mi/hr"), tol=1, rel=True)

In [10]:
speed.dimensionality

<UnitsContainer({'[length]': 1, '[time]': -1})>

In [11]:
speed.units

<Unit('mile / hour')>

In [12]:
speed.ito("km/s")

4.4704 m/s (nom), 0 km/s (lb), 8.9408 m/s (ub)